In [13]:
import pandas as pd
import numpy as np

In [38]:
demand = np.array([30,40,30])
supply = np.array([15,15,70])

In [39]:
#gen_data = np.random.randint(1, 10, len(supply)*len(demand))
gen_data = ([1,2,6],
            [5,3,4],
            [4,5,7])

In [40]:
costs = np.reshape(gen_data, (-1, len(demand)))
my_costs = costs.copy
costs

array([[1, 2, 6],
       [5, 3, 4],
       [4, 5, 7]])

In [41]:
def illstr_table(dmnd, sply):
    i_table = pd.DataFrame(costs, columns=dmnd, index=sply)
    for i in range(len(dmnd)):
        i_table.rename(columns={'0': str(dmnd[i])})
    return i_table

In [42]:
illstr_table(demand, supply) #illustration

,30,40,30
15,1,2,6
15,5,3,4
70,4,5,7


In [43]:
def check_balanced(dmnd, sply):
    return "Balanced" if sum(dmnd)==sum(sply) else "Unbalanced"

In [44]:
check_balanced(demand, supply)

'Balanced'

In [45]:
display(illstr_table(demand, supply).replace(costs, '-'))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


In [46]:
def change_row_v_for_illstr(t, i):
    for j in range(len(t.columns)):
        if t.values[i][j] == '-': 
            t.iat[i,j] = '+'

def change_column_v_for_illstr(t, j):
    for i in range(len(t.index)):
         if t.values[i][j] == '-': 
                t.iat[i,j] = '+'
                
def solution(dmnd, sply):  
    fg = 0
    cap_coord = []
    table = illstr_table(dmnd, sply).replace(costs, '-')
    display(table)
    for i in range(len(sply)):
        for j in range(len(dmnd)):
            if j != fg:
                continue
            fg += 1
            cell_v = min(sply[i], dmnd[j])
            sply[i] -= cell_v
            dmnd[j] -= cell_v
            table.index = sply
            table.columns = dmnd
            cap_coord.append([cell_v, [i,j]])
            if sply[i] == 0:
                fg = j
                change_row_v_for_illstr(table, i)
                table.iat[i,j] = cell_v
                display(table)
                break
            else: 
                change_column_v_for_illstr(table, j)
                table.iat[i,j] = cell_v
                display(table)
    return cap_coord

In [47]:
def get_ans(cap_coord, csts):
    summ=0
    route = []
    for cap, coord in cap_coord:
        summ += csts[coord[0]][coord[1]]*cap
        route.append(coord)
    return summ, route

In [48]:
cost, route = get_ans(solution(demand, supply), costs)
print("Cost: {}, \nRoute: {}".format(cost, route))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


,15,40,30
0,15,+,+
15,-,-,-
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,0,-,-


,0,0,30
0,15,+,+
0,15,+,+
30,0,40,-


,0,0,0
0,15,+,+
0,15,+,+
0,0,40,30


Cost: 500, 
Route: [[0, 0], [1, 0], [2, 0], [2, 1], [2, 2]]


In [52]:
def find(target,ToSearch):
    x=[(i, t.index(target))
    for i, t in enumerate(ToSearch)
    if target in t]
    return x

def MatMin(supplyinfo,demandinfo,costinfo):
    C=np.matrix(costinfo.copy())
    sn=len(supplyinfo)
    dn=len(demandinfo)
    sol=np.zeros((sn,dn))
    costs=list(costinfo)
    supply=supplyinfo.copy()
    demand=demandinfo.copy()
    count=0
    while (sum(supply)+sum(demand)) != 0 and count<20:
        count=count+1
        Min=[]
        for a in range(sn):
            row=costs[a]
            Row=[]
            for b in range(dn):
                if type(row[b])==int:
                    Row.append(row[b])
                 
                if b==dn-1 and len(Row)<=0:
                    continue
                if b==dn-1:
                    Min.append(min(Row))
        if len(Min)==0:
            return sol
        smallest=min(Min)
        location=find(smallest,costs)
        if len(location)==1: 
            location=find(smallest,costs)[0]
            costs[location[0]][location[1]]='x'
            if supply[location[0]]>=demand[location[1]]:   
                sol[location[0]][location[1]]=demand[location[1]]
                demand[location[0]]=supply[location[0]]-demand[location[1]]
                demand[location[1]]=0
            else:
                sol[location[0]][location[1]]=supply[location[0]]
                demand[location[1]]=demand[location[1]]-supply[location[0]]
                supply[location[0]]=0 
                
        else: 
            locations=find(smallest,costs)
            l=len(locations)
            m=[]
            for a in range(l):
                L=locations[a]
                s=L[1]
                d=L[0]
                if s>=d:
                    m.append(s)
                else:
                    m.append(d)
            M=max(m)
            loc=m.index(M)
            location=locations[loc]
            costs[location[0]][location[1]]='x'
            if supply[location[0]]>=demand[location[1]]:                
                sol[location[0]][location[1]]=demand[location[1]]
                supply[location[0]]=supply[location[0]]-demand[location[1]]
                demand[location[1]]=0     

            else:
                sol[location[0]][location[1]]=supply[location[0]]
                demand[location[1]]=demand[location[1]]-supply[location[0]]
                supply[location[0]]=0 

    feasabilitytest=np.count_nonzero(sol)
    print('Number of stepping stones =', feasabilitytest)
    if feasabilitytest==(sn+dn-1):
        print('Feasibility test passed')
    print(sol)
    z=0
    for k in range(sn):
        for l in range(dn):
            z=z+C[k,l]*sol[k,l]
    print('Cost is £', int(z),'for this solution.')
                
                

costs=[[1,2,6],[5,3,4],[4,5,7]]
MatMin([15,15,70],[30,40,30],costs)

array([[15.,  0.,  0.],
       [ 0., 15.,  0.],
       [15., 25., 45.]])

In [5]:
# Demans - A,B,C,D
# W,X,Y - Supply
# Cost matrix is given below
import collections
costs  = {'W': {'A': 11, 'B': 13, 'C': 17, 'D': 14},
          'X': {'A': 16, 'B': 18, 'C': 14, 'D': 10},
          'Y': {'A': 21, 'B': 24, 'C': 13, 'D': 10}}
demand = {'A': 200, 'B': 225, 'C': 275, 'D': 250}
cols = sorted(iter(demand.keys()))
supply = {'W': 250, 'X': 300, 'Y': 400}
res = dict((k, defaultdict(int)) for k in costs)
g = {}

#g contains all the columns and rows names like - a,b,c,... and so on

for x in supply:
    g[x] = sorted(costs[x].iterkeys(), key=lambda g: costs[x][g])
for x in demand:
    g[x] = sorted(costs.iterkeys(), key=lambda g: costs[g][x])

while g:
    d = {}
    for x in demand:
        d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x]) if len(g[x]) > 1 else costs[g[x][0]][x]
    s = {}
    for x in supply:
        s[x] = (costs[x][g[x][1]] - costs[x][g[x][0]]) if len(g[x]) > 1 else costs[x][g[x][0]]
    f = max(d, key=lambda n: d[n])
    t = max(s, key=lambda n: s[n])
    t, f = (f, g[f][0]) if d[f] > s[t] else (g[t][0], t)
    v = min(supply[f], demand[t])
    res[f][t] += v
    demand[t] -= v
    if demand[t] == 0:
        for k, n in supply.iteritems():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del demand[t]
    supply[f] -= v
    if supply[f] == 0:
        for k, n in demand.iteritems():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del supply[f]
 
for n in cols:
    print ("\t", n)
print()
cost = 0
for g in sorted(costs):
    print (g, "\t")
    for n in cols:
        y = res[g][n]
        if y != 0:
            print (y)
        cost += y * costs[g][n]
        print ("\t")
    print()
print ("\n\nTotal Cost = ", cost)

NameError: name 'defaultdict' is not defined